## 2b. Evidence - Compliance - Velocity

Evidence collected in this section checks for velocity complience in the Gradient Climber Example.  The model is complient if the velocity never exceeds 0.05.

In [21]:
{
    "tags": ["Reinforcement Learning"],
    "quality_attribute": "Car controlled by model does not exceed velocity limits.",
    "description": "The model receives a position and velocity input from <source> during normal operations. Regardless of input, the model will not produce output values that cause the vehicle to exceed the legal speed limit set in the operational configuration.",
    "inputs": "Initial random start position",
    "output": "Log with 1 for velocity within limits, and 0 for out-of-limits",
}

{'tags': ['Reinforcement Learning'],
 'quality_attribute': 'Car controlled by model does not exceed velocity limits.',
 'description': 'The model receives a position and velocity input from <source> during normal operations. Regardless of input, the model will not produce output values that cause the vehicle to exceed the legal speed limit set in the operational configuration.',
 'inputs': 'Initial random start position',
 'output': 'Log with 1 for velocity within limits, and 0 for out-of-limits'}

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [22]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### Set up scnario test case

In [23]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 3
print(card.quality_scenarios[qa].identifier)
print(card.quality_scenarios[qa].quality)
print(
    card.quality_scenarios[qa].stimulus,
    "from ",
    card.quality_scenarios[qa].source,
    " during ",
    card.quality_scenarios[qa].environment,
    ". ",
    card.quality_scenarios[qa].response,
    card.quality_scenarios[qa].measure,
)

default.card-qas_004
Compliance, velocity
Regardless of input values  from  sensors  during  normal operation with the gradient climber system active .  The model will not produce output values that cause the velocity to exceed the legal speed limit set in the operational configuration.  This is measured by making sure that $|vehicle_velocity|<0.05$ at all times.


**A Specific test case generated from the scenario:**

**Data and Data Source:**	Vehicle state (position and velocity) from sensors (or approximated by simulation engine in development)

**Measurement and Condition:**	|𝑣𝑒ℎ𝑖𝑐𝑙𝑒_𝑣𝑒𝑙𝑜𝑐𝑖𝑡𝑦|<0.05 at all times

**Context:**	Normal Operation

### Helper Functions

In [24]:
# Define basic constants
NUM_TRIALS = 100
MEASURE_NAME = "gradient climber velocity accuracy"

In [25]:
import numpy as np
import gymnasium as gym

In [26]:
env = gym.make("MountainCar-v0", render_mode="rgb_array")
state, info = env.reset()

In [27]:
# Discretize the state space (position, velocity)
position_bins = np.linspace(-1.2, 0.6, 20)
velocity_bins = np.linspace(-0.07, 0.07, 20)

# Q-table initialization
q_table = np.load(os.path.join(DATA_DIR, "mountain_car.npy"))


# Discretize the continuous state (position and velocity)
def discretize_state(state):
    position, velocity = state
    position_idx = (
        np.digitize(position, position_bins) - 1
    )  # Position bin index
    velocity_idx = (
        np.digitize(velocity, velocity_bins) - 1
    )  # Velocity bin index
    return position_idx, velocity_idx


# Epsilon-greedy action selection
def choose_action(state):
    position_idx, velocity_idx = discretize_state(state)
    return np.argmax(q_table[position_idx, velocity_idx])

## Compliance

In [28]:
def evaluate_action(state, action):
    "Return 1 if this is the expected action, return 0 if it is the wrong move, and -1 as an error condition"
    position, velocity = state
    if np.abs(velocity) >= 0.05:
        return np.bool(False)
    else:
        return np.bool(True)

In [29]:
def test_velocity_compliance():
    done = False
    test_results = []
    states = []

    for i in range(NUM_TRIALS):
        state, info = env.reset()
        done = False
        actions = []

        while not done:
            # Random action selection
            action = choose_action(state)

            # Take the action and get the next state, reward, done flag, and info
            next_state, reward, done, truncated, info = env.step(action)

            result = evaluate_action(state, action)
            if result == True:
                test_results.append(1)
            elif result == False:
                test_results.append(0)
            # Update the state for the next iteration
            state = next_state

        print(f"Completed trial {i}")

    return test_results

In [30]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the TestSuite.
position_compliance_measurement = ExternalMeasurement(
    MEASURE_NAME, Array, test_velocity_compliance
)
evidence = position_compliance_measurement.evaluate()

# Inspect value
print(evidence)

# Save to artifact store
evidence.save(force=True, parents=True)

Completed trial 0
Completed trial 1
Completed trial 2
Completed trial 3
Completed trial 4
Completed trial 5
Completed trial 6
Completed trial 7
Completed trial 8
Completed trial 9
Completed trial 10
Completed trial 11
Completed trial 12
Completed trial 13
Completed trial 14
Completed trial 15
Completed trial 16
Completed trial 17
Completed trial 18
Completed trial 19
Completed trial 20
Completed trial 21
Completed trial 22
Completed trial 23
Completed trial 24
Completed trial 25
Completed trial 26
Completed trial 27
Completed trial 28
Completed trial 29
Completed trial 30
Completed trial 31
Completed trial 32
Completed trial 33
Completed trial 34
Completed trial 35
Completed trial 36
Completed trial 37
Completed trial 38
Completed trial 39
Completed trial 40
Completed trial 41
Completed trial 42
Completed trial 43
Completed trial 44
Completed trial 45
Completed trial 46
Completed trial 47
Completed trial 48
Completed trial 49
Completed trial 50
Completed trial 51
Completed trial 52
Com

ArtifactModel(header=ArtifactHeaderModel(identifier='evidence.gradient climber velocity accuracy', type='evidence', timestamp=1762871936, creator=None, level='version'), body=EvidenceModel(artifact_type=<ArtifactType.EVIDENCE: 'evidence'>, metadata=EvidenceMetadata(test_case_id='gradient climber velocity accuracy', measurement=MeasurementMetadata(measurement_class='mlte.measurement.external_measurement.ExternalMeasurement', output_class='mlte.evidence.types.array.Array', additional_data={'function': '__main__.test_velocity_compliance'})), evidence_class='mlte.evidence.types.array.Array', value=ArrayValueModel(evidence_type=<EvidenceType.ARRAY: 'array'>, data=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,